In [2]:
pip install pyldpc

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from pyldpc import make_ldpc, encode, decode, get_message
from utils import *
import matplotlib.pyplot as plt
from statsmodels.tsa.api import VAR
import json
from statsmodels.tsa.ar_model import AutoReg
import numpy as np
from sympy import Matrix
import json
import argparse
from math import log

In [4]:
pip install matlabengine

Note: you may need to restart the kernel to use updated packages.


In [5]:
import matlab.engine

In [6]:
eng = matlab.engine.start_matlab()

### Functions to be used 

In [3]:
def generate_random_symbols(n):
    # Generate n random bits as -1 or 1 using uniform sampling.
    return np.random.choice([-1, 1], size=n)


def transmit_data(data, channel, snr_db):
    # Apply channel gain to the transmitted data.
    signal = channel * data
    # Calculate the signal power.
    signal_power = np.mean(np.abs(signal)**2)
    # Convert SNR from dB to linear scale and compute noise power.
    noise_power = signal_power / (10**(snr_db / 10))
    # Generate complex Gaussian noise.
    noise = np.sqrt(noise_power / 2) * (np.random.randn(len(data)) + 1j * np.random.randn(len(data)))
    # Add noise to the signal to create the received signal.
    return signal + noise


def transmit_data_awgn(data, snr_db):
    # Apply channel gain to the transmitted data.
    signal = data
    # Calculate the signal power.
    signal_power = np.mean(np.abs(signal)**2)
    # Convert SNR from dB to linear scale and compute noise power.
    noise_power = signal_power / (10**(snr_db / 10))
    # Generate complex Gaussian noise.
    noise = np.sqrt(noise_power / 2) * (np.random.randn(len(data)) + 1j * np.random.randn(len(data)))
    # Add noise to the signal to create the received signal.
    return signal + noise


def estimate_channel(received, data):
    # Reshape the transmitted data for matrix operations.
    X = data.reshape(-1, 1)
    # Perform least squares estimation to solve for the channel.
    h = np.linalg.lstsq(X, received, rcond=None)[0][0]
    return h


def number_of_symbols(N, n, k):
    T = N // (k)
    #This is the number of symbols to be transmitted
    N1 = T*n
    return int(N1)


def decoder_custom_awgn(received):
    list1 = np.sign(np.real(received))
    list2 = []
    for i in list1:
        if i == -1:
            list2.append(1)
        if i == 1:
            list2.append(0)
    return list2


def decoder_custom_jakes(received, channel):
    list1 = np.sign(np.real(received / channel))
    list2 = []
    for i in list1:
        if i == -1:
            list2.append(1)
        if i == 1:
            list2.append(0)
    return list2


def generate_channels_for_doppler_frequencies(Fd, Fs, N):
    def jakes_sos(P, K, Fs, Fd, N, typ):
        t = np.linspace(0, P/Fs, P)
        omega_d = 2 * np.pi * Fd
        
        # Initialize jakes_rvs to store real or complex numbers
        jakes_rvs = np.zeros((K, P), dtype=complex)
        
        for k in range(K):
            alpha = np.random.uniform(0, 2 * np.pi, N)
            alpha_m = np.array([((2 * np.pi * n) - np.pi + al)/(4*N) for n, al in enumerate(alpha)])
            a_m = np.random.uniform(0, 2 * np.pi, N)
            b_m = np.random.uniform(0, 2 * np.pi, N)
            
            cosine_terms = np.cos((omega_d * t[:, None] * np.cos(alpha_m)) + a_m)
            real_part = np.sqrt(1/N) * np.sum(cosine_terms, axis=1)
            
            if typ == 'comp':
                sine_terms = np.sin((omega_d * t[:, None] * np.cos(alpha_m)) + b_m)
                imag_part = np.sqrt(1/N) * np.sum(sine_terms, axis=1)
                jakes_rvs[k] = real_part + 1j * imag_part
            else:
                jakes_rvs[k] = real_part + 1j * 0 # Enforcing the real numbers also to be modelled as complex for easy use
        
        return jakes_rvs
    P = N
    K = 1
    typ = 'comp'
    jakes_rvs = jakes_sos(P, K, Fs, Fd, 100, typ)
    correlated_jakes_rvs = jakes_rvs
    # Open a file to write the generated channels
    with open("channels.txt", "w") as fo:
        # Iterate through each sample
        for i in range(P):
            # Extract the i-th sample
            new_sample = correlated_jakes_rvs[:, i]
            # Serialize complex numbers as (real, imag) pairs
            new_sample_serialized = [(z.real, z.imag) for z in new_sample]
            # Write serialized sample to the file
            json.dump(new_sample_serialized, fo)
            # Add a newline after each serialized sample
            fo.write('\n')
    # Return the generated Jakes random variables
    return correlated_jakes_rvs


### Simulation of the pipeline for transmission

In [4]:
Fd = 100
Fs = 10000000
N1 = 10000000
generate_channels_for_doppler_frequencies(Fd, Fs, N1)

array([[-0.56375683+0.81880986j, -0.56341028+0.81828215j,
        -0.56306364+0.81775435j, ...,  0.2861337 -0.99780321j,
         0.28601234-0.99749417j,  0.285891  -0.99718502j]])